### Group similar motivations

We are going to regrabbed the motivations from the AI


In [ ]:
# CELL 2
import os
import json
from datetime import datetime
import pytz

promt_file = 'prompt_no_guidance.txt'

# Define the prompt for each individual request
def prompt_formatted() -> str:    
    # Read the first file and set a string variable
    with open(promt_file, 'r') as file:
        prompt = file.read()
        
    with open('prompt_issues.md', 'r') as file:
        issues = file.read()

    with open('prompt_fact_sheet.md', 'r') as file:
        fact_sheet = file.read()

    prompt = prompt.replace('|issues|', issues)
    # prompt = prompt.replace('|guidance_note|', guidance_note)
    prompt = prompt.replace('|fact_sheet|', fact_sheet)    

    return prompt

def get_submission(submission_text: str, submission_author: str):
    prompt = ""
    prompt += "\n\n***************************************** SUBMISSION START *****************************************\n\n"

    prompt += f"Submission from: {submission_author}\n\n"
    
    prompt += submission_text

    prompt += "\n\n***************************************** SUBMISSION END *****************************************\n\n"
    return prompt

def get_function():
    with open('function_motivations.json', 'r') as f:
        function = json.load(f)
    return function

def get_file_path(doc_id, folder_path = './data/files'):    
    for file_name in os.listdir(folder_path):
        if file_name.startswith(doc_id):
            return os.path.join(folder_path, file_name)

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

md_file_location = './data/files'

file_counter = 0
jsonl_file = f"./data/cleaning/toProcess/jsonl_{file_counter}.jsonl"

skipped_files = []

# Make cleaning directories
os.makedirs('./data/cleaning', exist_ok=True)
os.makedirs('./data/cleaning/toProcess', exist_ok=True)
os.makedirs('./data/cleaning/output', exist_ok=True)

prompt = prompt_formatted()

# This loop takes each submission and adds it to the jsonl file in a format that can be used by the OpenAI API
for i in list["data"]:
    if i["metadata"]["removed"] == "Y":
        skipped_files.append(i["uniqueId"])
        continue
    try:
        md_file_path = get_file_path(i.get("uniqueId"))       
        with open(md_file_path, 'r') as file:
            submission = file.read()                
        sub_author = i["submitter"]
        submission_formatted = get_submission(submission, sub_author)
        function = get_function()

        ldata = { "custom_id": i["uniqueId"], "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-2024-05-13", "messages": [{"role": "system", "content": prompt},{"role": "user", "content": submission_formatted}], "max_tokens": 4096, "temperature": 1e-9, "frequency_penalty": 0, "presence_penalty": 0, "top_p": 0, "tools":[function], "tool_choice": { "type": "function", "function": { "name": "submission_eval" } }}}        
        
        if os.path.exists(jsonl_file) and os.path.getsize(jsonl_file) >= 85 * 1024 * 1024:  # 
            file_counter += 1
            jsonl_file = f"./data/cleaning/toProcess/jsonl_{file_counter}.jsonl"
        
        i["metadata"]["step_1"]["redo_motivation"] = {"batch_file": f'jsonl_{file_counter}.jsonl'}
        
        with open(jsonl_file, 'a') as f:
            json.dump(ldata, f)
            f.write('\n')

    except Exception as e:
        print(e)
        continue

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list["metadata"]["step_1"]["redo_motivations"] = {
    "fn": get_function(),    
    "time": current_time
}

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

In [ ]:
# CELL 3
from openai import OpenAI
import os
from datetime import datetime
import pytz

client = OpenAI(api_key=os.getenv('OPENAI_KEY'), max_retries=3)

jsonl_dir = './data/cleaning/toProcess'

jsonl_files = [f for f in os.listdir(jsonl_dir) if os.path.isfile(os.path.join(jsonl_dir, f)) and f.endswith('.jsonl')]

file_ids = []

for file in jsonl_files:
    file_object = client.files.create(
        file=open(f"{jsonl_dir}/{file}", "rb"),
        purpose="batch"
    )
    file_ids.append(file_object.id)

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list["metadata"]["step_1"]["redo_motivations"]["file_upload"] = {
            "timestamp": current_time,
            "file_ids": file_ids,
        }

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

In [ ]:
# We have now uploaded all the files and have their IDs, lets create a batch job for each
batch_ids = []

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

file_ids = list["metadata"]["step_1"]["redo_motivations"]["file_upload"]["file_ids"]

for file_id in file_ids:
    job = client.batches.create(
            input_file_id=file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
          )
    batch_ids.append(job.id)

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list["metadata"]["step_1"]["redo_motivations"]["batch_creation"] = {
            "timestamp": current_time,
            "batch_ids": batch_ids,
        }

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

In [ ]:
with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

desired_batch_ids = list["metadata"]["step_1"]["redo_motivations"]["batch_creation"]["batch_ids"]

batch_jobs = client.batches.list()

for batch in batch_jobs.data:
    if batch.id in desired_batch_ids:
        print(batch.id, batch.status, batch.request_counts)

In [ ]:

batch_jobs = client.batches.list()

with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

# we only want to download the batch jobs that were set up in cell 11
desired_batch_ids = list["metadata"]["step_1"]["redo_motivations"]["batch_creation"]["batch_ids"]

success_count = 0
error_count = 0

created_time = None
completion_time = None

for batch in batch_jobs.data:
    if batch.id in desired_batch_ids:
        # Gets the first created batch time
        if not created_time or batch.created_at < created_time:
            created_time = batch.created_at
        # Gets the lasted completed batch time
        if not completion_time or batch.completed_at > completion_time:
            completion_time = batch.completed_at
        if batch.output_file_id:
            success_count += batch.request_counts.total - batch.request_counts.failed
            output_file = batch.output_file_id
            content = client.files.content(output_file)
            jsonl_file_path = f'./data/cleaning/output/{output_file}.jsonl'
            content.write_to_file(jsonl_file_path)
        # Handle error files
        if batch.error_file_id:
            error_count += batch.request_counts.failed
            err_file = batch.error_file_id
            err_content = client.files.content(err_file)
            err_jsonl_file_path = f'./data/cleaning/output/err_{err_file}.jsonl'
            err_content.write_to_file(err_jsonl_file_path)

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

time_difference = completion_time - created_time

list["metadata"]["step_1"]["redo_motivations"]["batch_creation"]["download_timestamp"] = current_time
list["metadata"]["step_1"]["redo_motivations"]["batch_creation"]["completetion_duration_seconds"] = time_difference
list["metadata"]["step_1"]["redo_motivations"]["batch_creation"]["success"] = success_count
list["metadata"]["step_1"]["redo_motivations"]["batch_creation"]["errors"] = error_count

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

In [ ]:
import pandas as pd

# Parses the JSON from a function call, if there is an error in JSON parsing, recalls the LLM with the fix json function to get a valid json response.
def parse_JSON(json_str: str) -> dict:        
    try: 
        return json.loads(json_str)
    except Exception as e:              
        messages = [
      {
        'role': 'system',
        'content':
          'Assistant is a large language model designed to fix and return correct JSON objects.',
      },
      {
        'role': 'user',
        'content': f'ORIGINAL ERROR CONTAINING JSON OBJECT:\n\n{json_str}\n\nERROR MESSAGE: {e}',
      },
    ]
        
        tool_choices = [{
      'type': 'function',
      'function': {
        'name': 'fix_object',
        'description':
          'You will be given an incorrectly formed JSON Object and a error message. You must fix the incorrect JSON Object and return the valid JSON object.',
        'parameters': {
          'type': 'object',
          'properties': {
            'fixedJSON': {
              'type': 'string',
              'description': 'The reformated and error free JSON object. Return the JSON object only!',
            },
          },
          'required': ['fixedJSON'],
        },
      },
    }]                
        response = client.chat.completions.create(
                    model='gpt-4o-2024-05-13',
                    messages=messages,                    
                    max_tokens=4096,
                    temperature=0,
                    tools=tool_choices,
                    tool_choice={ 'type': 'function', 'function': { 'name': 'fix_object' } },        
                )        
                
        second_test_json = response.choices[0].message.tool_calls[0].function.arguments 
                  
        to_return = json.loads(second_test_json)
        return json.loads(to_return['fixedJSON'])

output_folder = './data/cleaning/output'

jsonl_files = [f for f in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, f)) and f.endswith('.jsonl')]

# Load original JSON list
with open('./data/step1/list.json', 'r') as f:
    list_data = json.load(f)

prompt_tokens = 0
completion_tokens = 0
total_tokens = 0

# Load the JSONL files
for file in jsonl_files:    
    with open(f"{output_folder}/{file}", "r") as f:
        for line in f:
            item = json.loads(line)            
            item_key = item['custom_id']            
            # grab the matching item in our list            
            list_item = next((x for x in list_data["data"] if x['uniqueId'] == item_key), None)
            if list_item:
              if item["response"]["status_code"] != 200:                
                list_item["metadata"]["step_1"]["redo_motivation"]["error"] = item["response"]
                continue
              json_res = parse_JSON(item['response']['body']['choices'][0]['message']['tool_calls'][0]['function']['arguments'])
              list_item["step_1"]["new_motivations"] = json_res
              list_item["metadata"]["step_1"]["redo_motivation"]["system_fingerprint"] = item['response']['body']['system_fingerprint']
              list_item["metadata"]["step_1"]["redo_motivation"]["batch_id"] = item['id']
              prompt_tokens += item['response']['body']["usage"]["prompt_tokens"]
              completion_tokens += item['response']['body']["usage"]["completion_tokens"]
              total_tokens += item['response']['body']["usage"]["total_tokens"]

local_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z")

list_data["metadata"]["step_1"]["redo_motivations"]["batch_processed"] = {"timestamp": current_time}
list_data["metadata"]["step_1"]["redo_motivations"]["usage"] = {"prompt_tokens": prompt_tokens, "completion_tokens": completion_tokens, "total_tokens": total_tokens}

# Save the updated list back to the json file
with open('./data/step1/list.json', 'w') as f:
    json.dump(list_data, f)

# Export the list to an Excel file for review
# Convert JSON to DataFrame
df = pd.json_normalize(list_data["data"])

# Save DataFrame to Excel
df.to_excel('./data/cleaning/step1.xlsx', index=False)

## Lets go reclustering!

#### This first cell purely updates all motivations to lowercase strings & saves a list of all motivations


In [ ]:
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
import json

# Data to process
with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

all_motivations = []

for item in tqdm(list['data']):
    if "step_1" not in item or item['step_1'] is None or "new_motivations" not in item['step_1']:        
        continue
    try:
        motivations = item['step_1']['new_motivations']['motivations'] if 'motivations' in item['step_1']['new_motivations'] else []
        if not motivations:            
            continue
        if len(motivations) == 0:            
            continue
        lower_case_motivations = []
        for motivation in motivations:
            motivation['label'] = motivation['label'].lower()
            lower_case_motivations.append(motivation)
        item['step_1']['new_motivations']['motivations'] = lower_case_motivations
        all_motivations.extend(lower_case_motivations)
        if 'updated_motivations' in item['metadata']['step_1']:
            item['metadata']['step_1']['updated_motivations']['new_motivations'] = {'original': motivations}
        else:
            item['metadata']['step_1']['updated_motivations'] = {'new_motivations': {'original': motivations}}
        continue
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")        
        continue

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

with open('data_cleaning_motivations_new.json', 'w') as file:
    json.dump(all_motivations, file)

### We are going to cluster similar motivations


In [1]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, SpectralClustering
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from kneed import KneeLocator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from ai_client import get_vector
import os
import warnings

## Get embeddings for all unique list items

This is done in parallel to speed up processing of large lists


In [12]:
# import json
# from concurrent.futures import ThreadPoolExecutor

# # Load the JSON file
# with open('./data/cleaning/data_cleaning_motivations_new.json', 'r') as file:
#     data = json.load(file)

# def get_embedding(string, main_pbar):    
#     vector = get_vector(string)
#     main_pbar.update(1)
#     return vector

# def load_saved_embeddings(file_path):
#     if os.path.exists(file_path):
#         return np.load(file_path, allow_pickle=True).item()
#     return None

# # Generate embeddings for all unique topics
# def generate_embeddings_parallel(data, file_path='./data/cleaning/data_cleaning_motivations_new_embeddings.npy'):
#     # Check if embeddings are already saved
#     saved_embeddings = load_saved_embeddings(file_path)
#     if saved_embeddings is not None:
#         print("Embeddings loaded from file.")
#         return saved_embeddings
    
#     embeddings_dict = {}
#     with tqdm(total=len(data), desc="Overall Progress") as main_pbar:
#         with ThreadPoolExecutor(max_workers=5) as executor:
#             futures = {executor.submit(get_embedding, item['explanation'], main_pbar): item['label'] for item in data}
#             for future in futures:
#                 label = futures[future]
#                 embeddings_dict[label] = future.result()
    
#     np.save(file_path, embeddings_dict)
#     return embeddings_dict

# embeddings = generate_embeddings_parallel(data)
# topics = [item['label'] for item in data]

import json
from concurrent.futures import ThreadPoolExecutor

# Load the JSON file
with open('./data/cleaning/data_cleaning_motivations_new.json', 'r') as file:
    data = json.load(file)

def get_embedding(string, main_pbar):    
    vector = get_vector(string)
    main_pbar.update(1)
    return vector

def load_saved_embeddings(file_path):
    if os.path.exists(file_path):
        return np.load(file_path, allow_pickle=True).item()
    return None

# Generate embeddings for all unique topics
def generate_embeddings_parallel(data, file_path='./data/cleaning/data_cleaning_motivations_new_embeddings_labels.npy'):
    # Check if embeddings are already saved
    saved_embeddings = load_saved_embeddings(file_path)
    if saved_embeddings is not None:
        print("Embeddings loaded from file.")
        return saved_embeddings
    
    embeddings_dict = {}
    with tqdm(total=len(data), desc="Overall Progress") as main_pbar:
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(get_embedding, item['label'], main_pbar): item['label'] for item in data}
            for future in futures:
                label = futures[future]
                embeddings_dict[label] = future.result()
    
    np.save(file_path, embeddings_dict)
    return embeddings_dict

embeddings = generate_embeddings_parallel(data)
topics = [item['label'] for item in data]

Overall Progress: 100%|██████████| 6460/6460 [05:22<00:00, 20.05it/s]


In [13]:
embeddings_file_path = './data/cleaning/data_cleaning_motivations_new_embeddings_labels.npy'
embeddings_dict = np.load(embeddings_file_path, allow_pickle=True).item()
labels, embeddings = zip(*embeddings_dict.items())
embeddings = np.array(embeddings)

## New approach to determining best cluster number and algorithm


In [16]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args

number_iterations = 100

# Define the search space for the number of clusters
space_kmeans = [Integer(500, 1500, name='n_clusters')]

# Function to compute silhouette score for KMeans with a given number of clusters
@use_named_args(space_kmeans)
def objective_kmeans(**params):
    kmeans = KMeans(n_clusters=params['n_clusters'], random_state=0, n_init=10)
    labels = kmeans.fit_predict(embeddings)
    if len(set(labels)) > 1:
        score = silhouette_score(embeddings, labels)
        print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score={score}")
        return -score  # gp_minimize minimizes the objective
    else:
        print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score=-1 (invalid clustering)")
        return -1

# Optimize using gp_minimize
res_kmeans = gp_minimize(objective_kmeans, space_kmeans, n_calls=number_iterations, random_state=0)

# Extract the best number of clusters and fit the final model
best_n_clusters_kmeans = res_kmeans.x[0]
optimal_kmeans = KMeans(n_clusters=best_n_clusters_kmeans, random_state=0, n_init=10)
kmeans_labels = optimal_kmeans.fit_predict(embeddings)

print(f"Optimal number of clusters for KMeans: {best_n_clusters_kmeans}")
print(f"Silhouette score for optimal KMeans clustering: {-res_kmeans.fun}")

Evaluated n_clusters=1093, silhouette_score=0.11369458188183214
Evaluated n_clusters=1344, silhouette_score=0.10523331843048692
Evaluated n_clusters=1358, silhouette_score=0.1039372689484801
Evaluated n_clusters=1347, silhouette_score=0.10325736442105107
Evaluated n_clusters=1124, silhouette_score=0.113735079746472
Evaluated n_clusters=884, silhouette_score=0.11753883855143994
Evaluated n_clusters=798, silhouette_score=0.11568954816411134
Evaluated n_clusters=557, silhouette_score=0.11189430385791628
Evaluated n_clusters=773, silhouette_score=0.11884258445102616
Evaluated n_clusters=978, silhouette_score=0.11579700389034121
Evaluated n_clusters=1500, silhouette_score=0.09265232164377508
Evaluated n_clusters=500, silhouette_score=0.107479443836351
Evaluated n_clusters=683, silhouette_score=0.11120513532728288
Evaluated n_clusters=898, silhouette_score=0.11695964787638678
Evaluated n_clusters=975, silhouette_score=0.11808259182474293
Evaluated n_clusters=900, silhouette_score=0.117033033

/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [900] before, using random point [1148]
  warnings.warn(


Evaluated n_clusters=1148, silhouette_score=0.11418016752202426
Evaluated n_clusters=907, silhouette_score=0.11840117059070629


/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [907] before, using random point [780]
  warnings.warn(


Evaluated n_clusters=780, silhouette_score=0.11334857554416014
Evaluated n_clusters=933, silhouette_score=0.117684914716962


...

Evaluated n_clusters=1220, silhouette_score=0.11247055472677138


/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [875] before, using random point [785]
  warnings.warn(


Evaluated n_clusters=785, silhouette_score=0.11529545262994637
Evaluated n_clusters=880, silhouette_score=0.11756976169137393
Evaluated n_clusters=881, silhouette_score=0.11808071928716952


/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [881] before, using random point [1324]
  warnings.warn(


Evaluated n_clusters=1324, silhouette_score=0.10506249453847463
Optimal number of clusters for KMeans: 926
Silhouette score for optimal KMeans clustering: 0.12147910555197793


In [17]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from sklearn_extra.cluster import KMedoids
import numpy as np
from sklearn.metrics import silhouette_score
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args

number_iterations = 100

# Define the search space for each algorithm
# space_gmm = [Integer(500, 1500, name='n_components')]
# space_agglomerative = [Integer(500, 1500, name='n_clusters')]
# space_spectral = [Integer(500, 1500, name='n_clusters')]
space_kmedoids = [Integer(500, 1500, name='n_clusters')]

# Objective functions for each algorithm
# @use_named_args(space_gmm)
# def objective_gmm(**params):
#     gmm = GaussianMixture(n_components=params['n_components'], random_state=0)
#     labels = gmm.fit_predict(embeddings)
#     if len(set(labels)) > 1:
#         score = silhouette_score(embeddings, labels)
#         print(f"Evaluated n_components={params['n_components']}, silhouette_score={score}")
#         return -score
#     else:
#         print(f"Evaluated n_components={params['n_components']}, silhouette_score=-1 (invalid clustering)")
#         return -1

# @use_named_args(space_agglomerative)
# def objective_agglomerative(**params):
#     agglomerative = AgglomerativeClustering(n_clusters=params['n_clusters'])
#     labels = agglomerative.fit_predict(embeddings)
#     if len(set(labels)) > 1:
#         score = silhouette_score(embeddings, labels)
#         print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score={score}")
#         return -score
#     else:
#         print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score=-1 (invalid clustering)")
#         return -1

# @use_named_args(space_spectral)
# def objective_spectral(**params):
#     spectral = SpectralClustering(n_clusters=params['n_clusters'], affinity='nearest_neighbors', random_state=0)
#     labels = spectral.fit_predict(embeddings)
#     if len(set(labels)) > 1:
#         score = silhouette_score(embeddings, labels)
#         print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score={score}")
#         return -score
#     else:
#         print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score=-1 (invalid clustering)")
#         return -1

@use_named_args(space_kmedoids)
def objective_kmedoids(**params):
    kmedoids = KMedoids(n_clusters=params['n_clusters'], random_state=0)
    labels = kmedoids.fit_predict(embeddings)
    if len(set(labels)) > 1:
        score = silhouette_score(embeddings, labels)
        print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score={score}")
        return -score
    else:
        print(f"Evaluated n_clusters={params['n_clusters']}, silhouette_score=-1 (invalid clustering)")
        return -1

# Optimize using gp_minimize for each algorithm
# res_gmm = gp_minimize(objective_gmm, space_gmm, n_calls=number_iterations, random_state=0)
# res_agglomerative = gp_minimize(objective_agglomerative, space_agglomerative, n_calls=number_iterations, random_state=0)
# res_spectral = gp_minimize(objective_spectral, space_spectral, n_calls=number_iterations, random_state=0)
res_kmedoids = gp_minimize(objective_kmedoids, space_kmedoids, n_calls=number_iterations, random_state=0)

# Extract the best parameters and fit the final models
# best_n_components_gmm = res_gmm.x[0]
# optimal_gmm = GaussianMixture(n_components=best_n_components_gmm, random_state=0)
# gmm_labels = optimal_gmm.fit_predict(embeddings)

# best_n_clusters_agglomerative = res_agglomerative.x[0]
# optimal_agglomerative = AgglomerativeClustering(n_clusters=best_n_clusters_agglomerative)
# agglomerative_labels = optimal_agglomerative.fit_predict(embeddings)

# best_n_clusters_spectral = res_spectral.x[0]
# optimal_spectral = SpectralClustering(n_clusters=best_n_clusters_spectral, affinity='nearest_neighbors', random_state=0)
# spectral_labels = optimal_spectral.fit_predict(embeddings)

best_n_clusters_kmedoids = res_kmedoids.x[0]
# best_n_clusters_kmedoids = 1431
optimal_kmedoids = KMedoids(n_clusters=best_n_clusters_kmedoids, random_state=0)
kmedoids_labels = optimal_kmedoids.fit_predict(embeddings)

# print(f"Optimal parameters for GMM: n_components={best_n_components_gmm}")
# print(f"Silhouette score for optimal GMM clustering: {-res_gmm.fun}")

# print(f"Optimal parameters for Agglomerative: n_clusters={best_n_clusters_agglomerative}")
# print(f"Silhouette score for optimal Agglomerative clustering: {-res_agglomerative.fun}")

# print(f"Optimal parameters for Spectral: n_clusters={best_n_clusters_spectral}")
# print(f"Silhouette score for optimal Spectral clustering: {-res_spectral.fun}")

print(f"Optimal parameters for KMedoids: n_clusters={best_n_clusters_kmedoids}")
print(f"Silhouette score for optimal KMedoids clustering: {-res_kmedoids.fun}")


Evaluated n_clusters=1093, silhouette_score=0.004155804847657322
Evaluated n_clusters=1344, silhouette_score=0.0038946658276575886
Evaluated n_clusters=1358, silhouette_score=0.004784171096741816
Evaluated n_clusters=1347, silhouette_score=0.0048614323478317345
Evaluated n_clusters=1124, silhouette_score=0.006046221521312991
Evaluated n_clusters=884, silhouette_score=0.008195342049264365
Evaluated n_clusters=798, silhouette_score=0.006733843718994157
Evaluated n_clusters=557, silhouette_score=0.009942557881647248
Evaluated n_clusters=773, silhouette_score=0.006776188653818977
Evaluated n_clusters=978, silhouette_score=0.01029828427009118
Evaluated n_clusters=641, silhouette_score=0.011049255320260852
Evaluated n_clusters=605, silhouette_score=0.009736384088298527
Evaluated n_clusters=1500, silhouette_score=0.0004493817740684819
Evaluated n_clusters=500, silhouette_score=0.010018859828558414
Evaluated n_clusters=1226, silhouette_score=0.0048578217380362354


/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [500] before, using random point [1377]
  warnings.warn(


Evaluated n_clusters=1377, silhouette_score=0.006933437450266583
Evaluated n_clusters=699, silhouette_score=0.00578000035967275
Evaluated n_clusters=941, silhouette_score=0.010478157421894542
Evaluated n_clusters=527, silhouette_score=0.008515517209699558
Evaluated n_clusters=631, silhouette_score=0.012497530350974489
Evaluated n_clusters=629, silhouette_score=0.013618183438542038
Evaluated n_clusters=625, silhouette_score=0.012436280930415164
Evaluated n_clusters=1014, silhouette_score=0.00955827405666176
Evaluated n_clusters=1430, silhouette_score=0.004516393302999053
Evaluated n_clusters=1282, silhouette_score=0.003111933160049257
Evaluated n_clusters=1174, silhouette_score=0.006504165515405454
Evaluated n_clusters=842, silhouette_score=0.006605473892801467
Evaluated n_clusters=1048, silhouette_score=0.006805089437641608
Evaluated n_clusters=916, silhouette_score=0.0082041333598851
Evaluated n_clusters=737, silhouette_score=0.007456869613433635
Evaluated n_clusters=627, silhouette_s

/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [500] before, using random point [1495]
  warnings.warn(


Evaluated n_clusters=1495, silhouette_score=0.0010578146326994137


...

Evaluated n_clusters=544, silhouette_score=0.009088335320285934


/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [630] before, using random point [890]
  warnings.warn(


Evaluated n_clusters=890, silhouette_score=0.00825777835672612


/Users/k/Documents/Local Development/03. UTS/new-acma-power-consultation/.venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [500] before, using random point [838]
  warnings.warn(


Evaluated n_clusters=838, silhouette_score=0.00592702576161626
Optimal parameters for KMedoids: n_clusters=630
Silhouette score for optimal KMedoids clustering: 0.013814934278241002


## From the above cell, set the best label value

It will be one of:

- kmeans_labels
- dbscan_labels
- gmm_labels


In [18]:
# best_labels = kmeans_labels
# best_algorithm = 'kmeans' # or anything else (as KMedoids is only value that changes code execution)

# kmedoids = 1431
# kmeans = 926

optimal_kmeans_num = 926
optimal_kmedoids_num = 630

## Group and display the topics

This will output the list of clusters


In [19]:
# Organize clustered topics
clustered_topics = [[] for _ in range(optimal_kmeans_num)]
for topic, label in zip(topics, kmeans_labels):
    clustered_topics[label].append(topic)

# Prepare the dictionary to save
clusters_dict = {}

    # For other algorithms, calculate the centroid and find the closest topic
for i, cluster in enumerate(clustered_topics):
    # Get the embeddings for the current cluster
    cluster_embeddings = np.array([embeddings[topics.index(topic)] for topic in cluster])
    
    # Calculate the centroid of the cluster
    centroid = np.mean(cluster_embeddings, axis=0)
    
    # Calculate the distances from each embedding to the centroid
    distances = np.linalg.norm(cluster_embeddings - centroid, axis=1)
    
    # Find the index of the representative topic (the one closest to the centroid)
    representative_idx = np.argmin(distances)
    
    # Add the representative topic and the full list of topics to the dictionary
    clusters_dict[f"{cluster[representative_idx]}"] = {
        "topics": cluster
    }

# Save to a JSON file
with open('./data/cleaning/data_cleaning_motivations_new_clusters_kmeans.json', 'w') as json_file:
    json.dump(clusters_dict, json_file, indent=4)

print("Clusters have been saved to clusters.json")


Clusters have been saved to clusters.json


In [20]:
import numpy as np
import json
# Import the KMedoids class
from sklearn_extra.cluster import KMedoids

# Assume embeddings is a numpy array of your topic embeddings and optimal_k is defined
# Fit the k-medoids model
# kmedoids = KMedoids(n_clusters=optimal_k, random_state=0).fit(embeddings)

# Organize clustered topics based on k-medoids labels
clustered_topics = [[] for _ in range(optimal_kmedoids_num)]
for topic, label in zip(topics, kmedoids_labels):
    clustered_topics[label].append(topic)

# Prepare the dictionary to save
clusters_dict = {}

# For k-medoids, directly use the medoid as the representative topic
for i, cluster in enumerate(clustered_topics):
    # The index of the medoid in the original embeddings array
    medoid_idx = optimal_kmedoids.medoid_indices_[i]
    
    # The topic corresponding to the medoid
    medoid_topic = topics[medoid_idx]
    
    # Add the representative topic (medoid) and the full list of topics to the dictionary
    clusters_dict[f"{medoid_topic}"] = {
        "topics": cluster
    }

# Save to a JSON file
with open('./data/cleaning/data_cleaning_motivations_new_clusters_kmedoids.json', 'w') as json_file:
    json.dump(clusters_dict, json_file, indent=4)

print("Clusters have been saved to clusters.json")


Clusters have been saved to clusters.json


## Visualise clusters

Not necessary but to visualise the results continue here


In [ ]:
perplexity_value = min(50, len(topics)-1)
tsne = TSNE(n_components=2, perplexity=20, learning_rate=500, max_iter=2000, 
            early_exaggeration=50, init='pca')
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(10, 10))
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=best_labels, cmap='tab20', alpha=0.6)

for i, cluster in enumerate(clustered_topics):
    centroid_2d = np.mean(embeddings_2d[best_labels == i], axis=0)
    plt.text(centroid_2d[0], centroid_2d[1], str(i+1), color='black', fontsize=9, ha='center', va='center')

plt.title('Cluster Visualization in 2D Space')
plt.colorbar(scatter)
plt.grid(True)
plt.show()

### We now update the list.json with the clusters


In [ ]:
with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

with open('data_cleaning_motivations_clusters.json', 'r') as f:
    clusters = json.load(f)

with tqdm(total=len(list["data"]), desc="Overall Progress") as main_pbar:
    for item in list["data"]:
        if "step_1" not in item or item['step_1'] is None:
            main_pbar.update(1)
            continue
        motivations = item['step_1']['motivations'] if 'motivations' in item['step_1'] else []
        if (len(motivations) == 0):
            main_pbar.update(1)
            continue        
        new_motivations = []
        for motivation in motivations:
            for cluster, values in clusters.items():
                if motivation == cluster:
                    if cluster not in new_motivations:
                        new_motivations.append(cluster)
                elif motivation in values['topics']:
                    if cluster not in new_motivations:
                        new_motivations.append(cluster)
        item['step_1']['motivations'] = new_motivations
        item['metadata']['step_1']['updated_motivations']['clustered'] = new_motivations
        main_pbar.update(1)

with open('./data/step1/list.json', 'w') as f:
    json.dump(list, f)

### Lets test to see if new clusters have been applied


In [ ]:
import json

# Load the JSON file
with open('./data/step1/list.json', 'r') as file:
    data = json.load(file)

# Initialize an empty list
topics_all = []

for item in data["data"]:
    if "step_1" not in item or item['step_1'] is None:
        continue
    motivations = item['step_1']['motivations'] if 'motivations' in item['step_1'] else []
    for motivation in motivations:
        if motivation not in topics_all:
            topics_all.append(motivation)

print(f"Total unique motivations: {len(topics_all)}")

### Update the DB with changes


In [ ]:
from db.docs import DocumentManager
from db.db_instance import DBClient
from models.models import FilterItem
import json
from tqdm.notebook import tqdm
client = DBClient()
db = DocumentManager()

# Data to process
with open('./data/step1/list.json', 'r') as f:
    list = json.load(f)

for item in tqdm(list["data"]):
    if "step_1" not in item or item['step_1'] is None:
        continue
    motivations = item["metadata"]['step_1']['updated_motivations']["lower_case"] if 'updated_motivations' in item["metadata"]['step_1'] else []
    filter_item = FilterItem(property = 'uniqueId', value= item["uniqueId"], condition='equal')    
    document_id = db.search_docs_filtered([filter_item])    
    db.update_single_property(document_id[0]["uuid"], 'motivations', motivations)